# debug precompute

## Load data

In [1]:
from ipywidgets import interact
import os 
import nibabel as nib
import plotui
import numpy as np
root='alg_out/'

@interact
def selectData(dataset=['Task001_LiverTumor','Task002_Pancreas','Task003_Multiorgan']):
    @interact
    def fileviewer(file=sorted(os.listdir(f'{root}/{dataset}/GroundTruth'))):
        global gt,voxelsize
        gt_nib = nib.load(f'{root}/{dataset}/GroundTruth/{file}')
        gt=gt_nib.dataobj[...]
        from nibabel.affines import voxel_sizes
        voxelsize=voxel_sizes(gt_nib.affine)
        @interact
        def fileviewer2(typ=['CE','Dice','DiceCE','DiceFocal','DiceTopK10','TopK10']):
                global pr
                pr_nib = nib.load(f'{root}/{dataset}/{typ}/{file}')
                if (voxelsize!=voxel_sizes(pr_nib.affine)).any():
                    raise Error('voxel sizes are different')
                pr=pr_nib.dataobj[...]
                print(f'dataset {dataset} predict {typ} loaded....')
                

            

interactive(children=(Dropdown(description='dataset', options=('Task001_LiverTumor', 'Task002_Pancreas', 'Task…

In [ ]:
import importlib
import metric
importlib.reload(metric)

mme=metric.MME(gt.max()+1,debug={'show_precompute':0})
from auto_profiler import Profiler
with Profiler(filterExternalLibraries=False):
    helper=mme.precompute(gt,spacing=voxelsize)

'Time   [Hits * PerHit] Function name [Called from] [Function Location]'

Tree(nodes=(Node(name='root'),))

0


In [ ]:
import matplotlib.pyplot as plt

shape=gt[:,:,gt.sum(axis=0).sum(axis=0).argmax()].astype(np.uint8)
shape=gt
# shape=np.zeros((200,200),np.uint8)
# shape[10:90,10:50]=1

# shape[10:90,60:100]=1
import cc3d
marker=cc3d.connected_components(shape).astype(np.int8)
# marker=scipy.ndimage.find_objects(shape.astype(np.int8),3)
# plt.imshow(marker)


In [ ]:
from ipytree import Tree, Node
tree = Tree(stripes=True)
node2 = Node('node2',icon_style ='danger',icon='file')
tree.add_node(node2)
node3 = Node('node3', disabled=True)
node4 = Node('node4',show_icon =False)
node5 = Node('node5', [Node('1'), Node('2')])
node2.add_node(node3)
node2.add_node(node4)
node2.add_node(node5)

tree

In [ ]:
pip install -e git+https://github.com/modaresimr/auto_profiler.git#egg=auto_profiler

In [ ]:
import scipy
from auto_profiler import Timer
# with Timer.instance('dis'):
#     dst=scipy.ndimage.distance_transform_edt(shape>0)#+scipy.ndimage.distance_transform_edt(shape)
ddd=np.ones(marker.shape,np.uint8)

mark=-(marker)
inp=dst
with Timer.instance('watershed'):
    res=scipy.ndimage.watershed_ift(inp.astype(np.uint8), mark.astype(np.int8), structure=None, output=None)
res
# plt.imshow(-dst)

In [ ]:
res.min()

In [ ]:
mark


In [ ]:
marker.max()